In [1]:
# in addition to expected return, mean-variance optimisation, the basis for portfolio optimisation practice, 
# requires a risk model, a parameter for quantifying the asset risk. The covariance matrix is one of the 
# most important, a statistical entity that describes the volatility of assets returns and how they vary
# with one another. The principlle of diversification states that risk can be reduced by making many 
# uncorrelated bets, and this makes the covariance matrix a fundamental input for Markowitz-derived models

In [2]:
# as the case of returns, we also need to rely on historical data, one inevitable assumption in optimisation. 
# accordint to PyPortfolioOpt documentation, "the most straighforward approach is just calculate the sample 
# covariance matrix based on historical returns, but relatively recent (post-2000) research indicates that there 
# are much more robust statistical estimators of the covariance matrix. In addition to proving a wrapper around
# the estimators in sklearn, PyPortfolioOpt provides some experimental alternatives such as 
# semicovariance and exponentially weighted covariance". Let's explore with our portfolio of Brazilian stocks. 

In [3]:
# the risk_models module provides functions for estimating the covariance matrix taking as input 
# historical returns. 

In [10]:
import pandas as pd
import numpy as np
import cvxpy as cp
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import EfficientFrontier
from pypfopt import objective_functions
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from pypfopt import HRPOpt
from pypfopt import CLA
from pypfopt import black_litterman
from pypfopt import BlackLittermanModel
from pypfopt import plotting

In [11]:
df = pd.read_csv("br_stocks.csv", parse_dates=True, index_col="Date")
df.head()

,ITSA4,GOAU4,CMIG3,PETR4,TOTS3,USIM5,LAME4,BBAS3,CIEL3,LIGT3,ABEV3,KLBN11,VVAR3,CVCB3,GOLL4,EMBR3,CSNA3
Date,,,,,,,,,,,,,,,,,
2015-10-26,4.337310,2.629987,4.889023,7.282466,32.969135,2.948654,14.489809,12.885270,21.604973,12.372133,17.280500,19.230658,3.310584,13.152596,3.55,25.767944,4.115990
2015-10-27,4.301952,2.516463,4.774228,7.050539,33.044613,2.948654,14.449583,13.058755,21.803387,12.225429,17.394650,19.053841,3.310584,12.932006,3.56,26.999142,4.082797
2015-10-28,4.272487,2.450240,4.855263,7.375236,32.081654,2.822561,14.425447,12.837956,21.489256,12.117845,17.245377,18.854654,2.803557,13.400757,3.56,26.970060,4.016410
2015-10-29,4.207663,2.374557,4.747219,7.059817,31.430193,2.725565,13.572176,12.222869,20.557831,12.078724,16.683409,18.524618,2.475481,13.327228,3.49,27.183344,3.576597
2015-10-30,4.243021,2.459701,4.835006,7.152587,32.261009,2.696467,13.451403,12.593497,20.144411,12.548180,16.762438,18.558466,2.883091,13.419142,3.55,27.416006,3.601491


In [14]:
returns = expected_returns.returns_from_prices(df)
returns.head(10)

,ITSA4,GOAU4,CMIG3,PETR4,TOTS3,USIM5,LAME4,BBAS3,CIEL3,LIGT3,ABEV3,KLBN11,VVAR3,CVCB3,GOLL4,EMBR3,CSNA3
Date,,,,,,,,,,,,,,,,,
2015-10-27,-0.008152,-0.043165,-0.023480,-0.031847,0.002289,0.000000,-0.002776,0.013464,0.009184,-0.011858,0.006606,-0.009195,0.000000,-0.016772,0.002817,0.047780,-0.008064
2015-10-28,-0.006849,-0.026316,0.016973,0.046053,-0.029141,-0.042763,-0.001670,-0.016908,-0.014407,-0.008800,-0.008582,-0.010454,-0.153153,0.036247,0.000000,-0.001077,-0.016260
2015-10-29,-0.015172,-0.030888,-0.022253,-0.042767,-0.020306,-0.034365,-0.059150,-0.047912,-0.043344,-0.003228,-0.032587,-0.017504,-0.117021,-0.005487,-0.019663,0.007908,-0.109504
2015-10-30,0.008403,0.035857,0.018492,0.013141,0.026434,-0.010676,-0.008899,0.030323,-0.020110,0.038866,0.004737,0.001827,0.164659,0.006897,0.017192,0.008559,0.006960
2015-11-03,0.043055,0.080769,0.055866,0.099870,0.067309,0.053957,0.073010,0.088291,0.033654,0.009353,0.026716,0.057912,0.103448,0.027397,0.030986,0.040311,0.165899
2015-11-04,0.006658,-0.035587,0.030424,-0.047170,-0.028516,-0.006826,-0.011155,0.004028,-0.015353,0.027027,0.000000,-0.008621,-0.015625,0.006667,0.010929,-0.038069,-0.019763
2015-11-05,0.015873,0.000000,0.023106,0.003713,0.037257,-0.003437,0.021997,0.048710,0.020164,-0.030075,-0.007143,-0.017391,-0.123810,0.006623,0.027027,0.009541,-0.002016
2015-11-06,-0.023437,-0.036900,-0.066499,-0.035758,-0.005169,-0.044828,-0.043599,-0.049180,-0.012912,-0.034884,-0.025180,0.001770,0.000000,-0.032895,-0.047368,-0.016101,0.010101
2015-11-09,-0.020000,0.007663,-0.005376,0.011509,0.009298,-0.018050,-0.001153,-0.013793,-0.022423,0.020884,-0.014233,-0.015901,-0.003623,0.000000,-0.019337,-0.012807,0.012000


In [64]:
# the following command computes a covariance matrix using the risk model supplied in the method parameter
cov1 = risk_models.risk_matrix(df)
cov1

,ITSA4,GOAU4,CMIG3,PETR4,TOTS3,USIM5,LAME4,BBAS3,CIEL3,LIGT3,ABEV3,KLBN11,VVAR3,CVCB3,GOLL4,EMBR3,CSNA3
ITSA4,0.106266,0.094993,0.098203,0.104835,0.050166,0.104282,0.075074,0.123885,0.067584,0.087071,0.044329,0.023509,0.081927,0.093132,0.142570,0.055744,0.102228
GOAU4,0.094993,0.328751,0.139799,0.177492,0.071271,0.274447,0.103135,0.154817,0.087976,0.125427,0.056984,0.047403,0.140966,0.128543,0.217926,0.075680,0.257616
CMIG3,0.098203,0.139799,0.240716,0.139415,0.066079,0.159084,0.104854,0.148635,0.079999,0.159106,0.055980,0.031726,0.118915,0.126809,0.214762,0.069165,0.151079
PETR4,0.104835,0.177492,0.139415,0.277875,0.073124,0.191634,0.114993,0.172577,0.092070,0.119288,0.058783,0.045892,0.119123,0.137196,0.219356,0.080450,0.199364
TOTS3,0.050166,0.071271,0.066079,0.073124,0.240080,0.065111,0.065522,0.075296,0.072754,0.056432,0.036927,0.027804,0.074328,0.074532,0.108289,0.051181,0.076359
USIM5,0.104282,0.274447,0.159084,0.191634,0.065111,0.433124,0.115517,0.175969,0.102846,0.141380,0.056395,0.044733,0.152370,0.143734,0.270876,0.074938,0.294184
LAME4,0.075074,0.103135,0.104854,0.114993,0.065522,0.115517,0.179010,0.114198,0.079862,0.084227,0.053773,0.037566,0.129063,0.121231,0.157290,0.052131,0.115123
BBAS3,0.123885,0.154817,0.148635,0.172577,0.075296,0.175969,0.114198,0.227995,0.095646,0.131219,0.055649,0.030508,0.138464,0.143079,0.216086,0.073787,0.159577
CIEL3,0.067584,0.087976,0.079999,0.092070,0.072754,0.102846,0.079862,0.095646,0.223958,0.084380,0.052282,0.026414,0.091474,0.089493,0.135835,0.062857,0.097090
LIGT3,0.087071,0.125427,0.159106,0.119288,0.056432,0.141380,0.084227,0.131219,0.084380,0.287086,0.051788,0.025331,0.138131,0.133126,0.208813,0.069075,0.121305


In [60]:
# "check if a covariance matrix is positive semidefinite, and if not, fix it with the chosen method. 
# the spectral method sets negative eigenvalues to zero then rebuilds the matrix, while the diag method adds 
# a small positive value to the diagonal". 
cov1_fixed = risk_models.fix_nonpositive_semidefinite(cov1, fix_method='spectral')
cov1_fixed
# our portfolio's covariance matrix was positive and semidefinite. 

,ITSA4,GOAU4,CMIG3,PETR4,TOTS3,USIM5,LAME4,BBAS3,CIEL3,LIGT3,ABEV3,KLBN11,VVAR3,CVCB3,GOLL4,EMBR3,CSNA3
ITSA4,5472.180996,394.865825,385.024529,939.836044,328.393386,194.608461,-98.225250,174.807960,-81.859385,-66.455287,-106.631737,-107.889936,226.249059,-9.676364,93.510912,-56.173609,7.061673
GOAU4,394.865825,5157.682671,64.058955,-838.983802,350.542478,450.907411,72.462860,-242.233556,-85.246732,891.034841,-177.045717,-874.688238,4.281212,33.277938,45.980329,234.355854,964.393510
CMIG3,385.024529,64.058955,9805.618243,-2076.655109,-110.184604,358.445184,83.015502,-1213.618954,223.484169,721.057834,-89.055956,-0.628237,80.691613,1673.082387,-667.423536,-42.505734,116.673557
PETR4,939.836044,-838.983802,-2076.655109,21039.142837,-573.476435,142.508934,-162.559506,2436.821038,520.647092,-2102.477353,533.926242,1112.993244,14.865252,-166.126348,546.381277,-204.257574,492.325529
TOTS3,328.393386,350.542478,-110.184604,-573.476435,38420.058192,-278.761874,1121.794085,392.743395,309.381362,354.098588,-625.658456,209.607450,304.408842,333.653551,-793.177184,-443.641949,-126.255207
USIM5,194.608461,450.907411,358.445184,142.508934,-278.761874,8516.608834,-74.744308,1305.966800,252.159203,-32.002146,-224.409290,-86.268856,495.697368,449.981085,270.208433,-121.815221,460.629695
LAME4,-98.225250,72.462860,83.015502,-162.559506,1121.794085,-74.744308,14230.554811,383.458198,141.914153,445.074114,48.774291,460.744550,-8.351696,-394.583608,208.436148,1442.631947,81.062024
BBAS3,174.807960,-242.233556,-1213.618954,2436.821038,392.743395,1305.966800,383.458198,22161.199225,-357.121547,-779.013288,37.262316,110.499554,-583.165260,-38.890371,-113.342150,-138.363598,-521.783199
CIEL3,-81.859385,-85.246732,223.484169,520.647092,309.381362,252.159203,141.914153,-357.121547,6086.063839,301.258903,120.083976,-36.144872,545.788541,444.160218,301.570064,-37.831435,227.883684
LIGT3,-66.455287,891.034841,721.057834,-2102.477353,354.098588,-32.002146,445.074114,-779.013288,301.258903,17859.550318,-78.976808,-289.065642,-275.495600,250.990888,-85.222320,235.174004,726.876820


In [61]:
# the following command calculate the annualised sample covariance matrix of (daily) asset returns.

cov2 = risk_models.sample_cov(df)
cov2

,ITSA4,GOAU4,CMIG3,PETR4,TOTS3,USIM5,LAME4,BBAS3,CIEL3,LIGT3,ABEV3,KLBN11,VVAR3,CVCB3,GOLL4,EMBR3,CSNA3
ITSA4,5472.180996,394.865825,385.024529,939.836044,328.393386,194.608461,-98.225250,174.807960,-81.859385,-66.455287,-106.631737,-107.889936,226.249059,-9.676364,93.510912,-56.173609,7.061673
GOAU4,394.865825,5157.682671,64.058955,-838.983802,350.542478,450.907411,72.462860,-242.233556,-85.246732,891.034841,-177.045717,-874.688238,4.281212,33.277938,45.980329,234.355854,964.393510
CMIG3,385.024529,64.058955,9805.618243,-2076.655109,-110.184604,358.445184,83.015502,-1213.618954,223.484169,721.057834,-89.055956,-0.628237,80.691613,1673.082387,-667.423536,-42.505734,116.673557
PETR4,939.836044,-838.983802,-2076.655109,21039.142837,-573.476435,142.508934,-162.559506,2436.821038,520.647092,-2102.477353,533.926242,1112.993244,14.865252,-166.126348,546.381277,-204.257574,492.325529
TOTS3,328.393386,350.542478,-110.184604,-573.476435,38420.058192,-278.761874,1121.794085,392.743395,309.381362,354.098588,-625.658456,209.607450,304.408842,333.653551,-793.177184,-443.641949,-126.255207
USIM5,194.608461,450.907411,358.445184,142.508934,-278.761874,8516.608834,-74.744308,1305.966800,252.159203,-32.002146,-224.409290,-86.268856,495.697368,449.981085,270.208433,-121.815221,460.629695
LAME4,-98.225250,72.462860,83.015502,-162.559506,1121.794085,-74.744308,14230.554811,383.458198,141.914153,445.074114,48.774291,460.744550,-8.351696,-394.583608,208.436148,1442.631947,81.062024
BBAS3,174.807960,-242.233556,-1213.618954,2436.821038,392.743395,1305.966800,383.458198,22161.199225,-357.121547,-779.013288,37.262316,110.499554,-583.165260,-38.890371,-113.342150,-138.363598,-521.783199
CIEL3,-81.859385,-85.246732,223.484169,520.647092,309.381362,252.159203,141.914153,-357.121547,6086.063839,301.258903,120.083976,-36.144872,545.788541,444.160218,301.570064,-37.831435,227.883684
LIGT3,-66.455287,891.034841,721.057834,-2102.477353,354.098588,-32.002146,445.074114,-779.013288,301.258903,17859.550318,-78.976808,-289.065642,-275.495600,250.990888,-85.222320,235.174004,726.876820


In [87]:
# the following command estimate the semicovariance matrix, for instance, the covariance 
# given that the returns are less than the benchmark.

cov3 = risk_models.semicovariance(df)
cov3

,ITSA4,GOAU4,CMIG3,PETR4,TOTS3,USIM5,LAME4,BBAS3,CIEL3,LIGT3,ABEV3,KLBN11,VVAR3,CVCB3,GOLL4,EMBR3,CSNA3
ITSA4,0.039023,0.035816,0.038959,0.037956,0.021662,0.036996,0.030834,0.044364,0.026484,0.037571,0.019186,0.014082,0.034494,0.040917,0.056512,0.026279,0.035391
GOAU4,0.035816,0.121024,0.057731,0.070621,0.035856,0.093449,0.045367,0.060990,0.033504,0.053350,0.025724,0.022635,0.062277,0.060145,0.089214,0.037383,0.088729
CMIG3,0.038959,0.057731,0.087962,0.058585,0.029252,0.060683,0.048346,0.057453,0.033306,0.060133,0.024295,0.021146,0.049401,0.055458,0.085141,0.036099,0.058966
PETR4,0.037956,0.070621,0.058585,0.111855,0.035081,0.071930,0.051400,0.063942,0.034828,0.055665,0.027168,0.025789,0.060197,0.062151,0.091545,0.038673,0.072827
TOTS3,0.021662,0.035856,0.029252,0.035081,0.143414,0.030561,0.030723,0.033703,0.025885,0.028259,0.016153,0.015806,0.037471,0.042646,0.048918,0.029605,0.035633
USIM5,0.036996,0.093449,0.060683,0.071930,0.030561,0.129186,0.048617,0.066122,0.035836,0.056863,0.023983,0.023289,0.064853,0.063183,0.088160,0.038756,0.092088
LAME4,0.030834,0.045367,0.048346,0.051400,0.030723,0.048617,0.068517,0.049072,0.032587,0.043624,0.025104,0.020577,0.052556,0.059506,0.076602,0.030631,0.047375
BBAS3,0.044364,0.060990,0.057453,0.063942,0.033703,0.066122,0.049072,0.082896,0.035452,0.056286,0.024705,0.020385,0.057302,0.065202,0.087495,0.035864,0.059094
CIEL3,0.026484,0.033504,0.033306,0.034828,0.025885,0.035836,0.032587,0.035452,0.072550,0.035527,0.023198,0.016275,0.036289,0.047064,0.057211,0.031528,0.033221
LIGT3,0.037571,0.053350,0.060133,0.055665,0.028259,0.056863,0.043624,0.056286,0.035527,0.101404,0.023594,0.021436,0.062102,0.063531,0.082578,0.036807,0.046595


In [66]:
# the following command estimates the exponentially-weighted covariance matrix, which gives greater weight 
# to more recent data.

cov4 = risk_models.exp_cov(df)
cov4

,ITSA4,GOAU4,CMIG3,PETR4,TOTS3,USIM5,LAME4,BBAS3,CIEL3,LIGT3,ABEV3,KLBN11,VVAR3,CVCB3,GOLL4,EMBR3,CSNA3
ITSA4,0.146073,0.119612,0.127170,0.134018,0.092277,0.122423,0.089494,0.166507,0.137371,0.154206,0.088069,0.032609,0.139889,0.181699,0.201709,0.163548,0.127800
GOAU4,0.119612,0.306482,0.158456,0.215554,0.131889,0.277569,0.138053,0.169019,0.147203,0.182596,0.108499,0.090689,0.227828,0.226578,0.252744,0.190998,0.271569
CMIG3,0.127170,0.158456,0.222623,0.154128,0.126563,0.169139,0.123364,0.168734,0.166128,0.210811,0.107804,0.041333,0.189349,0.226288,0.248693,0.181187,0.171745
PETR4,0.134018,0.215554,0.154128,0.289852,0.133477,0.209698,0.145674,0.193257,0.162393,0.160505,0.111796,0.093053,0.212273,0.243151,0.291091,0.210732,0.214701
TOTS3,0.092277,0.131889,0.126563,0.133477,0.541228,0.120735,0.131468,0.140862,0.174033,0.119490,0.079407,0.054638,0.187247,0.158677,0.225747,0.116335,0.129206
USIM5,0.122423,0.277569,0.169139,0.209698,0.120735,0.382191,0.133342,0.182976,0.180281,0.213973,0.110134,0.088448,0.241162,0.244001,0.293905,0.221751,0.303839
LAME4,0.089494,0.138053,0.123364,0.145674,0.131468,0.133342,0.261472,0.137548,0.150179,0.113861,0.100960,0.072743,0.238968,0.226168,0.233529,0.120517,0.135932
BBAS3,0.166507,0.169019,0.168734,0.193257,0.140862,0.182976,0.137548,0.254030,0.179429,0.191569,0.107153,0.040178,0.228391,0.259965,0.297337,0.221369,0.179288
CIEL3,0.137371,0.147203,0.166128,0.162393,0.174033,0.180281,0.150179,0.179429,0.429211,0.196518,0.106079,0.034907,0.196199,0.209246,0.305468,0.191391,0.167078
LIGT3,0.154206,0.182596,0.210811,0.160505,0.119490,0.213973,0.113861,0.191569,0.196518,0.416559,0.117861,0.043581,0.238811,0.306418,0.323060,0.242342,0.201515


In [88]:
# the following command calculate minimum covariance determinant, an estimator of the covariance
# matrix that is more robust to noise, that is, more robusto to outliers and 'contaminated'data.
# an efficient estimator is implemented in the sklearn.covariance module, which is based on the 
# algorithm presented in Rousseauw (1999).

cov_to_corr = risk_models.cov_to_corr(cov1)
cov_to_corr

,ITSA4,GOAU4,CMIG3,PETR4,TOTS3,USIM5,LAME4,BBAS3,CIEL3,LIGT3,ABEV3,KLBN11,VVAR3,CVCB3,GOLL4,EMBR3,CSNA3
ITSA4,1.000000,0.508231,0.614010,0.610076,0.314078,0.486077,0.544316,0.795901,0.438091,0.498507,0.489696,0.220466,0.336067,0.496179,0.516253,0.377628,0.486888
GOAU4,0.508231,1.000000,0.496955,0.587247,0.253688,0.727309,0.425142,0.565488,0.324227,0.408274,0.357895,0.252745,0.328758,0.389360,0.448653,0.291483,0.697585
CMIG3,0.614010,0.496955,1.000000,0.539054,0.274874,0.492682,0.505116,0.634463,0.344548,0.605241,0.410887,0.197684,0.324098,0.448883,0.516700,0.311314,0.478091
PETR4,0.610076,0.587247,0.539054,1.000000,0.283113,0.552384,0.515595,0.685639,0.369072,0.422342,0.401570,0.266144,0.302179,0.452016,0.491200,0.337031,0.587192
TOTS3,0.314078,0.253688,0.274874,0.283113,1.000000,0.201917,0.316063,0.321833,0.313758,0.214952,0.271398,0.173477,0.202849,0.264182,0.260880,0.230674,0.241957
USIM5,0.486077,0.727309,0.492682,0.552384,0.201917,1.000000,0.414859,0.559974,0.330216,0.400938,0.308584,0.207793,0.309590,0.379305,0.485844,0.251458,0.694020
LAME4,0.544316,0.425142,0.505116,0.515595,0.316063,0.414859,1.000000,0.565270,0.398858,0.371540,0.457685,0.271435,0.407904,0.497637,0.438829,0.272099,0.422455
BBAS3,0.795901,0.565488,0.634463,0.685639,0.321833,0.559974,0.565270,1.000000,0.423274,0.512896,0.419694,0.195327,0.387765,0.520414,0.534193,0.341260,0.518880
CIEL3,0.438091,0.324227,0.344548,0.369072,0.313758,0.330216,0.398858,0.423274,1.000000,0.332776,0.397838,0.170632,0.258468,0.328431,0.338814,0.293318,0.318529
LIGT3,0.498507,0.408274,0.605241,0.422342,0.214952,0.400938,0.371540,0.512896,0.332776,1.000000,0.348069,0.144529,0.344731,0.431514,0.460029,0.284695,0.351505


In [89]:
stdevs = np.std(df)
stdevs

ITSA4      2.644850
GOAU4      2.179132
CMIG3      3.062689
PETR4      6.872585
TOTS3     12.464224
USIM5      3.003723
LAME4      4.998313
BBAS3     11.066040
CIEL3      6.414973
LIGT3      4.219119
ABEV3      2.252121
KLBN11     2.764380
VVAR3      4.333099
CVCB3     15.689143
GOLL4     10.566313
EMBR3      4.951796
CSNA3      3.524472
dtype: float64

In [90]:
# the following command converts a correlation matrix to a covariance matrix
corr_to_cov = risk_models.corr_to_cov(cov_to_corr,stdevs)
corr_to_cov

,ITSA4,GOAU4,CMIG3,PETR4,TOTS3,USIM5,LAME4,BBAS3,CIEL3,LIGT3,ABEV3,KLBN11,VVAR3,CVCB3,GOLL4,EMBR3,CSNA3
ITSA4,6.995234,2.929178,4.973700,11.089336,10.353906,3.861589,7.195748,23.294452,7.432931,5.562809,2.916883,1.611908,3.851457,20.589170,14.427376,4.945705,4.538626
GOAU4,2.929178,4.748617,3.316682,8.794763,6.890460,4.760605,4.630643,13.636378,4.532399,3.753675,1.756428,1.522521,3.104267,13.311732,10.330402,3.145285,5.357657
CMIG3,4.973700,3.316682,9.380067,11.346332,10.493055,4.532417,7.732462,21.503135,6.769360,7.820839,2.834113,1.673676,4.301089,21.569248,16.721107,4.721333,5.160687
PETR4,11.089336,8.794763,11.346332,47.232428,24.251893,11.403045,17.711380,52.144430,16.271436,12.246350,6.215461,5.056312,8.998769,48.738591,35.669909,11.469703,14.223099
TOTS3,10.353906,6.890460,10.493055,24.251893,155.356868,7.559587,19.690725,44.390308,25.087331,11.303893,7.618408,5.977297,10.955587,51.661615,34.358077,14.237240,10.629137
USIM5,3.861589,4.760605,4.532417,11.403045,7.559587,9.022350,6.228504,18.613163,6.362860,5.081116,2.087491,1.725391,4.029450,17.875087,15.419865,3.740146,7.347263
LAME4,7.195748,4.630643,7.732462,17.711380,19.690725,6.228504,24.983132,31.265950,12.789009,7.835203,5.152067,3.750478,8.834460,39.024308,23.176196,6.734616,7.442139
BBAS3,23.294452,13.636378,21.503135,52.144430,44.390308,18.613163,31.265950,122.457241,30.047541,23.946584,10.459641,5.975194,18.593412,90.352616,62.461732,18.699965,20.237328
CIEL3,7.432931,4.532399,6.769360,16.271436,25.087331,6.362860,12.789009,30.047541,41.151875,9.006768,5.747689,3.025881,7.184564,33.055104,22.965716,9.317421,7.201748
LIGT3,5.562809,3.753675,7.820839,12.246350,11.303893,5.081116,7.835203,23.946584,9.006768,17.800966,3.307342,1.685681,6.302327,28.563784,20.508348,5.947917,5.226944


In [91]:
# the risk_models module provides functions for estimating the covariance matrix given historical returns
# the format of the data input is the same as in :ref: 'expected_returns'
# currently the module is cabable of estimating: 1. fix non-positive semidefinite matrices; 2. general risk
# matrix function, allowing to run any risk model from one function; 3. sample covariance; 4. semicovariance;
# 5. exponentially weighted covariance; 6. minimum covariance determinant; 7. shrunk cov matrices, with 
# 7.1 manual shrinkage; 7.2 Ledoit Wolf shrinkage; 7.3 Oracle Approximating shrinkage;
# 8. covariance to correlation matrix

In [92]:
# the following code checks if a covariance matrix is positive semidefinite, and if not, fix with the chosen 
# method. spectral is a method that sets negative eigenvalues to zero oand then rebuilds the matrix, 
# while diag method adds a small positive value to the diagonal. 

nonpositive_check = risk_models.fix_nonpositive_semidefinite(cov1, fix_method='spectral')
nonpositive_check

,ITSA4,GOAU4,CMIG3,PETR4,TOTS3,USIM5,LAME4,BBAS3,CIEL3,LIGT3,ABEV3,KLBN11,VVAR3,CVCB3,GOLL4,EMBR3,CSNA3
ITSA4,0.106266,0.094993,0.098203,0.104835,0.050166,0.104282,0.075074,0.123885,0.067584,0.087071,0.044329,0.023509,0.081927,0.093132,0.142570,0.055744,0.102228
GOAU4,0.094993,0.328751,0.139799,0.177492,0.071271,0.274447,0.103135,0.154817,0.087976,0.125427,0.056984,0.047403,0.140966,0.128543,0.217926,0.075680,0.257616
CMIG3,0.098203,0.139799,0.240716,0.139415,0.066079,0.159084,0.104854,0.148635,0.079999,0.159106,0.055980,0.031726,0.118915,0.126809,0.214762,0.069165,0.151079
PETR4,0.104835,0.177492,0.139415,0.277875,0.073124,0.191634,0.114993,0.172577,0.092070,0.119288,0.058783,0.045892,0.119123,0.137196,0.219356,0.080450,0.199364
TOTS3,0.050166,0.071271,0.066079,0.073124,0.240080,0.065111,0.065522,0.075296,0.072754,0.056432,0.036927,0.027804,0.074328,0.074532,0.108289,0.051181,0.076359
USIM5,0.104282,0.274447,0.159084,0.191634,0.065111,0.433124,0.115517,0.175969,0.102846,0.141380,0.056395,0.044733,0.152370,0.143734,0.270876,0.074938,0.294184
LAME4,0.075074,0.103135,0.104854,0.114993,0.065522,0.115517,0.179010,0.114198,0.079862,0.084227,0.053773,0.037566,0.129063,0.121231,0.157290,0.052131,0.115123
BBAS3,0.123885,0.154817,0.148635,0.172577,0.075296,0.175969,0.114198,0.227995,0.095646,0.131219,0.055649,0.030508,0.138464,0.143079,0.216086,0.073787,0.159577
CIEL3,0.067584,0.087976,0.079999,0.092070,0.072754,0.102846,0.079862,0.095646,0.223958,0.084380,0.052282,0.026414,0.091474,0.089493,0.135835,0.062857,0.097090
LIGT3,0.087071,0.125427,0.159106,0.119288,0.056432,0.141380,0.084227,0.131219,0.084380,0.287086,0.051788,0.025331,0.138131,0.133126,0.208813,0.069075,0.121305


In [95]:
# the following code calculates the minimum covariance determinant, an estimator of the covariance
# matrix that is more robust to noise. the minimum covariance determinant estimator is designed to be robust
# to outliers and 'contaminated' data. 

min_cov_det = risk_models.min_cov_determinant(df)
min_cov_det

,ITSA4,GOAU4,CMIG3,PETR4,TOTS3,USIM5,LAME4,BBAS3,CIEL3,LIGT3,ABEV3,KLBN11,VVAR3,CVCB3,GOLL4,EMBR3,CSNA3
ITSA4,0.049787,0.032999,0.027357,0.035446,0.012725,0.032090,0.028670,0.047005,0.019885,0.019266,0.017929,0.013110,0.025924,0.025885,0.044179,0.017659,0.035450
GOAU4,0.032999,0.132164,0.031643,0.054353,0.017252,0.093214,0.032176,0.041862,0.020607,0.023651,0.016498,0.014848,0.025511,0.033152,0.043388,0.028497,0.099874
CMIG3,0.027357,0.031643,0.081314,0.034632,0.014654,0.038524,0.030094,0.039435,0.015922,0.030124,0.014370,0.013307,0.026860,0.033991,0.043952,0.015318,0.042248
PETR4,0.035446,0.054353,0.034632,0.084202,0.016018,0.053823,0.030481,0.049432,0.027492,0.023650,0.019083,0.013681,0.022739,0.029899,0.048159,0.018808,0.059471
TOTS3,0.012725,0.017252,0.014654,0.016018,0.061133,0.020053,0.015919,0.017194,0.012904,0.009507,0.007318,0.008412,0.015585,0.012995,0.022591,0.007987,0.015718
USIM5,0.032090,0.093214,0.038524,0.053823,0.020053,0.148980,0.044054,0.048063,0.023685,0.023523,0.018321,0.016757,0.031497,0.034542,0.056892,0.029023,0.106672
LAME4,0.028670,0.032176,0.030094,0.030481,0.015919,0.044054,0.072038,0.035903,0.022620,0.020032,0.015318,0.016911,0.039656,0.034901,0.047866,0.013670,0.038351
BBAS3,0.047005,0.041862,0.039435,0.049432,0.017194,0.048063,0.035903,0.077079,0.025137,0.027771,0.019978,0.013298,0.034023,0.034009,0.056797,0.018285,0.048600
CIEL3,0.019885,0.020607,0.015922,0.027492,0.012904,0.023685,0.022620,0.025137,0.095015,0.011456,0.014379,0.013297,0.022265,0.019283,0.034076,0.008528,0.026003
LIGT3,0.019266,0.023651,0.030124,0.023650,0.009507,0.023523,0.020032,0.027771,0.011456,0.071288,0.010416,0.009276,0.016573,0.023006,0.024929,0.011773,0.020187


In [96]:
# shrinkage estimators: the essential idea is that the unbiased but often poorly estimated sample covariance
# can be combined with a structured estimator, F, and a shrinkage constant which "shrinks" the sample cov
# matrix towards the other estimagor, the shrinkage target. this may be significantly biased but has little 
# estimation error. there are many possible options for the target, and each one will result in a different
# optimal shrinkage constant. PyPortfolioOpt offers two shrinkage methods: 
# Ledoit-Wolf and Oracle approximating shrinkage (OAS)

In [102]:
# the following code calculate the Ledoit-Wolf shrinkage estimate

sample_cov = risk_models.sample_cov(df, frequency=252)
S_ledoit_wolf = risk_models.CovarianceShrinkage(df).ledoit_wolf()
S_ledoit_wolf

,ITSA4,GOAU4,CMIG3,PETR4,TOTS3,USIM5,LAME4,BBAS3,CIEL3,LIGT3,ABEV3,KLBN11,VVAR3,CVCB3,GOLL4,EMBR3,CSNA3
ITSA4,0.111302,0.092292,0.095411,0.101854,0.048740,0.101317,0.072939,0.120362,0.065662,0.084595,0.043068,0.022840,0.079598,0.090484,0.138516,0.054158,0.099321
GOAU4,0.092292,0.327460,0.135824,0.172445,0.069244,0.266643,0.100203,0.150415,0.085475,0.121860,0.055363,0.046055,0.136958,0.124888,0.211730,0.073528,0.250290
CMIG3,0.095411,0.135824,0.241929,0.135451,0.064200,0.154560,0.101872,0.144409,0.077724,0.154582,0.054389,0.030824,0.115533,0.123203,0.208655,0.067198,0.146783
PETR4,0.101854,0.172445,0.135451,0.278031,0.071045,0.186185,0.111724,0.167670,0.089452,0.115896,0.057111,0.044587,0.115736,0.133295,0.213119,0.078163,0.193695
TOTS3,0.048740,0.069244,0.064200,0.071045,0.241310,0.063260,0.063659,0.073155,0.070685,0.054827,0.035877,0.027014,0.072215,0.072413,0.105210,0.049726,0.074187
USIM5,0.101317,0.266643,0.154560,0.186185,0.063260,0.428865,0.112232,0.170966,0.099921,0.137360,0.054791,0.043461,0.148037,0.139647,0.263173,0.072808,0.285819
LAME4,0.072939,0.100203,0.101872,0.111724,0.063659,0.112232,0.181978,0.110951,0.077591,0.081832,0.052244,0.036498,0.125393,0.117784,0.152817,0.050649,0.111849
BBAS3,0.120362,0.150415,0.144409,0.167670,0.073155,0.170966,0.110951,0.229569,0.092926,0.127488,0.054067,0.029641,0.134527,0.139010,0.209942,0.071689,0.155040
CIEL3,0.065662,0.085475,0.077724,0.089452,0.070685,0.099921,0.077591,0.092926,0.225647,0.081981,0.050795,0.025663,0.088873,0.086949,0.131972,0.061070,0.094329
LIGT3,0.084595,0.121860,0.154582,0.115896,0.054827,0.137360,0.081832,0.127488,0.081981,0.286980,0.050316,0.024611,0.134203,0.129341,0.202875,0.067111,0.117856


In [103]:
# the following code calculate the Oracle Approximating shrinkage estimate

sample_cov = risk_models.sample_cov(df, frequency=252)
S_OAS = risk_models.CovarianceShrinkage(df).oracle_approximating()
S_OAS

,ITSA4,GOAU4,CMIG3,PETR4,TOTS3,USIM5,LAME4,BBAS3,CIEL3,LIGT3,ABEV3,KLBN11,VVAR3,CVCB3,GOLL4,EMBR3,CSNA3
ITSA4,0.107205,0.094392,0.097582,0.104172,0.049849,0.103622,0.074599,0.123101,0.067156,0.086520,0.044048,0.023360,0.081409,0.092543,0.141667,0.055391,0.101581
GOAU4,0.094392,0.328281,0.138914,0.176369,0.070819,0.272710,0.102482,0.153837,0.087420,0.124633,0.056623,0.047103,0.140074,0.127729,0.216547,0.075201,0.255985
CMIG3,0.097582,0.138914,0.240804,0.138533,0.065661,0.158077,0.104190,0.147694,0.079493,0.158099,0.055626,0.031525,0.118162,0.126006,0.213403,0.068727,0.150123
PETR4,0.104172,0.176369,0.138533,0.277728,0.072662,0.190421,0.114266,0.171485,0.091487,0.118533,0.058410,0.045601,0.118369,0.136328,0.217968,0.079941,0.198102
TOTS3,0.049849,0.070819,0.065661,0.072662,0.240171,0.064699,0.065108,0.074819,0.072293,0.056075,0.036693,0.027628,0.073858,0.074061,0.107604,0.050857,0.075875
USIM5,0.103622,0.272710,0.158077,0.190421,0.064699,0.431994,0.114786,0.174855,0.102195,0.140486,0.056038,0.044450,0.151405,0.142824,0.269161,0.074464,0.292322
LAME4,0.074599,0.102482,0.104190,0.114266,0.065108,0.114786,0.179489,0.113475,0.079357,0.083694,0.053433,0.037329,0.128246,0.120464,0.156294,0.051801,0.114394
BBAS3,0.123101,0.153837,0.147694,0.171485,0.074819,0.174855,0.113475,0.228163,0.095041,0.130389,0.055297,0.030315,0.137587,0.142173,0.214719,0.073320,0.158567
CIEL3,0.067156,0.087420,0.079493,0.091487,0.072293,0.102195,0.079357,0.095041,0.224152,0.083846,0.051951,0.026247,0.090895,0.088927,0.134975,0.062459,0.096475
LIGT3,0.086520,0.124633,0.158099,0.118533,0.056075,0.140486,0.083694,0.130389,0.083846,0.286880,0.051461,0.025171,0.137257,0.132284,0.207491,0.068637,0.120537


In [104]:
# the following code shrinks a sample covariance matrix to the identity matrix (scaled by the average sample
# variance). this method does not estimate an optional shrinkage parameter, it requires manual input.
# we apply the default shrinkage parameter of 0.2

sample_cov = risk_models.sample_cov(df, frequency=252)
S_shrunk = risk_models.CovarianceShrinkage(df).shrunk_covariance(delta=0.2)
S_shrunk

,ITSA4,GOAU4,CMIG3,PETR4,TOTS3,USIM5,LAME4,BBAS3,CIEL3,LIGT3,ABEV3,KLBN11,VVAR3,CVCB3,GOLL4,EMBR3,CSNA3
ITSA4,0.143335,0.075994,0.078563,0.083868,0.040133,0.083425,0.060059,0.099108,0.054067,0.069657,0.035463,0.018807,0.065542,0.074506,0.114056,0.044595,0.081782
GOAU4,0.075994,0.321322,0.111839,0.141994,0.057016,0.219557,0.082508,0.123854,0.070381,0.100341,0.045587,0.037923,0.112773,0.102834,0.174341,0.060544,0.206092
CMIG3,0.078563,0.111839,0.250895,0.111532,0.052863,0.127267,0.083883,0.118908,0.063999,0.127285,0.044784,0.025381,0.095132,0.101447,0.171810,0.055332,0.120863
PETR4,0.083868,0.141994,0.111532,0.280622,0.058500,0.153307,0.091995,0.138062,0.073656,0.095430,0.047026,0.036713,0.095298,0.109757,0.175485,0.064360,0.159491
TOTS3,0.040133,0.057016,0.052863,0.058500,0.250385,0.052089,0.052418,0.060237,0.058203,0.045145,0.029542,0.022244,0.059463,0.059626,0.086631,0.040945,0.061087
USIM5,0.083425,0.219557,0.127267,0.153307,0.052089,0.404821,0.092414,0.140775,0.082277,0.113104,0.045116,0.035787,0.121896,0.114987,0.216700,0.059951,0.235347
LAME4,0.060059,0.082508,0.083883,0.091995,0.052418,0.092414,0.201530,0.091358,0.063890,0.067381,0.043019,0.030053,0.103251,0.096985,0.125832,0.041705,0.092098
BBAS3,0.099108,0.123854,0.118908,0.138062,0.060237,0.140775,0.091358,0.240718,0.076517,0.104975,0.044519,0.024407,0.110771,0.114463,0.172869,0.059030,0.127662
CIEL3,0.054067,0.070381,0.063999,0.073656,0.058203,0.082277,0.063890,0.076517,0.237488,0.067504,0.041826,0.021131,0.073179,0.071595,0.108668,0.050286,0.077672
LIGT3,0.069657,0.100341,0.127285,0.095430,0.045145,0.113104,0.067381,0.104975,0.067504,0.287990,0.041431,0.020265,0.110505,0.106501,0.167050,0.055260,0.097044
